# Simple place cell statistics for search and homing path maps

We calculate the statistics of firirng rate maps of the search and homing paths.

We also get the map stability between conditions.

In [1]:
%load_ext autoreload
%autoreload 2

%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/setup_project.py
%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/neuronAutopi.py

prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList,pose_file_extension = ".pose_kf.npy")

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1
dlcModelPath: /adata/models
Reading /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:22<00:00,  1.72it/s]


Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


Get intervals for the search and homing paths.

In [2]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 47.39it/s]


In [3]:
ses.intervalDict.keys()

dict_keys(['circ80', 'circ80_1', 'circ80_2', 'task', 'task_1', 'task_2', 'light', 'light_1', 'light_2', 'dark', 'dark_1', 'dark_2', 'searchPath_light', 'searchPath_light_1', 'searchPath_light_2', 'searchPath_dark', 'searchPath_dark_1', 'searchPath_dark_2', 'searchToLeverPath_light', 'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'homingPath_light', 'homingPath_light_1', 'homingPath_light_2', 'homingPath_dark', 'homingPath_dark_1', 'homingPath_dark_2', 'homingFromLeavingLever_light', 'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2', 'homingFromLeavingLeverToPeriphery_light', 'homingFromLeavingLeverToPeriphery_light_1', 'homingFromLeavingLeverToPeriphery_light_2', 'homingFromLeavingLeverToPeriphery_dark', 'homingFromLeavingLeverToPeriphery_dark_1', 'homingFromLeavingLeverToPeriphery_

In [4]:
def getMapStats(ses,sSes,onlyArena=False):
    """
    Get all the map stats of the data within the intervals of interest
    """
    
    xy_range=np.array([[-50,-90],[50,60]])
    
    # get all the conditions we are interested
    
    conditionDicts = [{"type" : "searchToLeverPath", "light" : "light"},
                      {"type" : "searchToLeverPath", "light" : "light_1"},
                      {"type" : "searchToLeverPath", "light" : "light_2"},
                      {"type" : "searchToLeverPath", "light" : "dark"},
                      {"type" : "searchToLeverPath", "light" : "dark_1"},
                      {"type" : "searchToLeverPath", "light" : "dark_2"},
                      {"type" : "homingFromLeavingLever", "light" : "light"},
                      {"type" : "homingFromLeavingLever", "light" : "light_1"},
                      {"type" : "homingFromLeavingLever", "light" : "light_2"},
                      {"type" : "homingFromLeavingLever", "light" : "dark"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_1"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_2"}]
    
    res = []
    for d in conditionDicts:
    
        navPathType= d["type"]
        light = d["light"]
    
        myDict = {}
        # we can now get intervals for any navPath types!
        inter = ses.intervalDict[navPathType+"_"+light]

        sSes.ap.set_intervals(inter)
        
        if onlyArena:
            # this should come after setting the time intervals, will be reset when calling sSes.ap.set_intervals again
            sSes.ap.invalid_outside_spatial_area(shape="circle",radius=43.0,center=np.array([0,0])) ## remove the bridge and homebase

        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 3, smoothing=True,xy_range=xy_range)
    
        # get peak rate from smoothed maps
        myDict["peakRate"] = [ np.nanmax(n.spatial_properties.firing_rate_map) for n in sSes.cg.neuron_list] # not affected by onlyArena!!!!
        myDict["meanRate"] = [ n.spike_train.mean_firing_rate() for n in sSes.cg.neuron_list]
        myDict["validBins"] = [ np.sum(~np.isnan(n.spatial_properties.firing_rate_map)) for n in sSes.cg.neuron_list] # not affected by onlyArena!!!!
        
        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing=False,xy_range=xy_range)
        
        myDict["info"] = [ n.spatial_properties.information_score() for n in sSes.cg.neuron_list]
        myDict["sparsity"] = [ n.spatial_properties.sparsity_score() for n in sSes.cg.neuron_list]
        myDict["condition"] =  navPathType+"_"+light
        myDict["session"] = sSes.name
        myDict["mouse"] = sSes.subject
        myDict["cellId"] = [sSes.name+"_"+n.name for n in sSes.cg.neuron_list]
        

        res.append(pd.DataFrame(myDict))
        
    # reset to original intervals
    sSes.ap.unset_intervals()
    
    return pd.concat(res)

## Try out on one session

In [5]:
ses = myProject.sessionList[0]
sSes = sSesList[0]

In [6]:
res = getMapStats(ses,sSes)

/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


In [7]:
res.head()

,peakRate,meanRate,validBins,info,sparsity,condition,session,mouse,cellId
0,47.284492,9.907041,194,0.400377,0.382117,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_80
1,30.461895,16.302290,194,0.444338,0.334698,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_90
2,53.661105,14.761577,194,0.302459,0.325100,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_92
3,13.225959,0.705801,194,3.925557,0.954621,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_96
4,13.343241,2.702703,194,0.550908,0.443964,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_98


## Run on all sessions

In [8]:
res = [ getMapStats(ses,sSes) for ses, sSes in tqdm(zip(myProject.sessionList,sSesList))]

0it [00:00, ?it/s]/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:660: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))
39it [07:25, 11.42s/it]


In [9]:
df = pd.concat(res)
df

,peakRate,meanRate,validBins,info,sparsity,condition,session,mouse,cellId
0,47.284492,9.907041,194,0.400377,0.382117,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_80
1,30.461895,16.302290,194,0.444338,0.334698,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_90
2,53.661105,14.761577,194,0.302459,0.325100,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_92
3,13.225959,0.705801,194,3.925557,0.954621,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_96
4,13.343241,2.702703,194,0.550908,0.443964,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_98
...,...,...,...,...,...,...,...,...,...
52,12.858016,1.886189,406,1.576006,0.729143,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_238
53,39.576939,14.993606,406,0.636290,0.468324,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_240
54,59.548147,3.793691,406,1.966183,0.873218,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_241
55,30.262844,4.038789,406,1.552495,0.777601,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_242


In [10]:
fn=myProject.dataPath+"/results/mySearchHomingMapStats.csv"
print(fn)
df.to_csv(fn,index=False)

/ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/mySearchHomingMapStats.csv


## Map similarity

We compare the map similarity across conditions.

In [15]:
from scipy.stats import pearsonr
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    if np.sum(indices)<2:
        return np.nan
    r,p = pearsonr(a[indices],b[indices])
    return r

In [16]:
fn = myProject.dataPath+"/results/mySearchHomingMaps.pickle" #, dictionary with key per condition
with open(fn, 'rb') as handle:
    myMaps = pickle.load(handle)

In [17]:
myMaps.keys()

dict_keys(['searchToLeverPath_light', 'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'homingFromLeavingLever_light', 'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2'])

In [19]:
myMaps['searchToLeverPath_light'].shape

(1080, 34, 50)

In [20]:
env1 = ['searchToLeverPath_light','searchToLeverPath_dark','searchToLeverPath_light','homingFromLeavingLever_light', "searchToLeverPath_light","searchToLeverPath_dark", "searchToLeverPath_light_1","searchToLeverPath_dark_1","homingFromLeavingLever_light_1","homingFromLeavingLever_dark_1"]
env2 = ['homingFromLeavingLever_light','homingFromLeavingLever_dark','searchToLeverPath_dark', 'homingFromLeavingLever_dark', "homingFromLeavingLever_dark", "homingFromLeavingLever_light", "searchToLeverPath_light_2","searchToLeverPath_dark_2","homingFromLeavingLever_light_2","homingFromLeavingLever_dark_2"]
condNames = ["slhl","sdhd","slsd","hlhd","slhd","sdhl","sl1sl2","sd1sd2","hl1hl2","hd1hd2"]
list(zip(env1,env2,condNames))

[('searchToLeverPath_light', 'homingFromLeavingLever_light', 'slhl'),
 ('searchToLeverPath_dark', 'homingFromLeavingLever_dark', 'sdhd'),
 ('searchToLeverPath_light', 'searchToLeverPath_dark', 'slsd'),
 ('homingFromLeavingLever_light', 'homingFromLeavingLever_dark', 'hlhd'),
 ('searchToLeverPath_light', 'homingFromLeavingLever_dark', 'slhd'),
 ('searchToLeverPath_dark', 'homingFromLeavingLever_light', 'sdhl'),
 ('searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'sl1sl2'),
 ('searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'sd1sd2'),
 ('homingFromLeavingLever_light_1',
  'homingFromLeavingLever_light_2',
  'hl1hl2'),
 ('homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2', 'hd1hd2')]

In [21]:
mapCorDf = pd.DataFrame()
nCells=myMaps[list(myMaps.keys())[0]].shape[0]
for e1,e2,c in zip(env1,env2,condNames):
    print(e1,e2,c)
    correlation  = [ map_cor(myMaps[e1][i],myMaps[e2][i]) for i in range(nCells) ]
    df1 = pd.DataFrame({"condition":c,
                       "r":correlation})
    mapCorDf = pd.concat([mapCorDf,df1])

searchToLeverPath_light homingFromLeavingLever_light slhl
searchToLeverPath_dark homingFromLeavingLever_dark sdhd
searchToLeverPath_light searchToLeverPath_dark slsd


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


homingFromLeavingLever_light homingFromLeavingLever_dark hlhd
searchToLeverPath_light homingFromLeavingLever_dark slhd
searchToLeverPath_dark homingFromLeavingLever_light sdhl


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


searchToLeverPath_light_1 searchToLeverPath_light_2 sl1sl2
searchToLeverPath_dark_1 searchToLeverPath_dark_2 sd1sd2
homingFromLeavingLever_light_1 homingFromLeavingLever_light_2 hl1hl2


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


homingFromLeavingLever_dark_1 homingFromLeavingLever_dark_2 hd1hd2


/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [22]:
mapCorDf

,condition,r
0,slhl,-0.281157
1,slhl,-0.364280
2,slhl,0.079883
3,slhl,0.161184
4,slhl,-0.602283
...,...,...
1075,hd1hd2,0.146109
1076,hd1hd2,0.298639
1077,hd1hd2,0.288522
1078,hd1hd2,-0.219147


In [23]:
mapCorDf.shape ## shoul not be 12680

(10800, 2)

In [24]:
fn=myProject.dataPath+"/results/searchHomingMapCorDf.csv"
print("Saving to",fn)
mapCorDf.to_csv(fn,index=False)

Saving to /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/searchHomingMapCorDf.csv
